In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.naive_bayes import *
from sklearn.ensemble import *
from sklearn.preprocessing import *

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

%pylab inline

pd.set_option('display.max_colwidth', 999)

Populating the interactive namespace from numpy and matplotlib


In [2]:
def logit_model(X_train,y_train,X_test,y_test,featurez):
    x_train_baseline = pd.DataFrame(x_train[featurez])
    x_test_baseline = pd.DataFrame(x_test[featurez])
    model = LogisticRegression()
    model.fit(x_train_baseline, y_train)
    y_pred = model.predict(x_test_baseline)
    print("accuracy i : "+ str(accuracy_score(y_pred, y_test)))
    print("precision i : "+ str(precision_score(y_pred, y_test)))
    print("recall i : "+ str(recall_score(y_pred, y_test)))
    

    fig,ax = plt.subplots(1,2, figsize = (5,5))    
    
    y_pred_prob = model.predict_proba(x_test_baseline)[:,1]
    fpr, tpr, all_thresholds = roc_curve(y_test, y_pred_prob)
#     hist(y_pred_prob)

    ax[0].hist(y_pred_prob, bins = 20)
    ax[1].plot(fpr,tpr)
    
    title('ROC Curve for Churn Prediction')
    grid(True)

In [3]:
def logit_model_loop(x_train,y_train,x_test,y_test,features_list):
    for index, featurez in enumerate(features_list):
        x_train_baseline = pd.DataFrame(x_train[featurez])
        x_test_baseline = pd.DataFrame(x_test[featurez])
        
        model = RandomForestClassifier(n_estimators= 36, min_samples_split= 18, min_samples_leaf = 24, max_depth = 15)
        model.fit(x_train_baseline, y_train)
        y_pred = model.predict(x_test_baseline)
        print("accuracy with model " + str(index) + ":  " + str(accuracy_score(y_pred, y_test)))
        print("precision with model " + str(index) + ":  " +  str(precision_score(y_pred, y_test)))
        print("recall with model " + str(index) + ":  " + str(recall_score(y_pred, y_test)))
        print(' ')

        y_pred_prob = model.predict_proba(x_test_baseline)[:,1]
        fpr, tpr, all_thresholds = roc_curve(y_test, y_pred_prob)
    
        figure = figsize(10,10)
        plot(fpr,tpr , label = 'model ' + str(index))
        legend()

        title('ROC Curve for Churn Prediction')
        grid(True)

In [4]:
df= pd.read_csv('data/churn_train.csv')
df['last_trip_date'] = pd.to_datetime(df['last_trip_date']) 

In [5]:
test_df= pd.read_csv('data/churn_test.csv')
test_df['last_trip_date'] = pd.to_datetime(test_df['last_trip_date']) 

In [6]:
df['LTV']=df['avg_surge']*df['trips_in_first_30_days']*df['avg_dist']
df['miles']=df['trips_in_first_30_days']*df['avg_dist']
df['avg_ride_price']=df['avg_surge']*df['avg_dist']

In [7]:
test_df['LTV']=test_df['avg_surge']*test_df['trips_in_first_30_days']*test_df['avg_dist']
test_df['miles']=test_df['trips_in_first_30_days']*test_df['avg_dist']
test_df['avg_ride_price']=test_df['avg_surge']*test_df['avg_dist']

In [8]:
df['target'] = (df['last_trip_date'] < '2014-06-01') * 1
df['constant'] = 1
df['luxury_car_user'] = (df['luxury_car_user'] == 0) * 1

In [9]:
test_df['target'] = (test_df['last_trip_date'] < '2014-06-01') * 1
test_df['constant'] = 1
test_df['luxury_car_user'] = (test_df['luxury_car_user'] == 0) * 1

In [10]:
df['avg_rating_of_driver_imputed'] = df['avg_rating_of_driver'].fillna(df.avg_rating_of_driver.median())

In [11]:
test_df['avg_rating_of_driver_imputed'] = test_df['avg_rating_of_driver'].fillna(df.avg_rating_of_driver.median())

In [12]:
df = df.join(pd.get_dummies(df['city']).join(df['target']).drop(['Winterfell','target'], 1))
df = df.join(pd.get_dummies(df['phone'], drop_first = True))

In [13]:
test_df = test_df.join(pd.get_dummies(test_df['city']).join(test_df['target']).drop(['Winterfell','target'], 1))
test_df = test_df.join(pd.get_dummies(test_df['phone'], drop_first = True))

In [14]:
# df['avg_dist'] = df.avg_dist.transform(lambda x: 50 if x > 50 else x)
# df['avg_ride_price'] = df.avg_ride_price.apply(lambda x: 50 if x > 50 else x )

In [15]:
df['avg_dist_log'] = np.log(1 + df['avg_dist'])
df['avg_ride_price_log'] = np.log(1 + df['avg_ride_price'])
df['avg_surge_log'] = np.log(1 + df['avg_surge'])
df['LTV_log'] = df['avg_surge_log']*df['trips_in_first_30_days']*df['avg_dist_log']
df['surge_pct_log'] = np.log(1 + df['surge_pct'])
df['weekday_pct_log'] = np.log(1 + df['weekday_pct'])
df['miles_log'] = np.log(1 + df['miles'])

In [16]:
test_df['avg_dist_log'] = np.log(1 + test_df['avg_dist'])
test_df['avg_ride_price_log'] = np.log(1 + test_df['avg_ride_price'])
test_df['avg_surge_log'] = np.log(1 + test_df['avg_surge'])
test_df['LTV_log'] = test_df['avg_surge_log']*test_df['trips_in_first_30_days']*test_df['avg_dist_log']
test_df['surge_pct_log'] = np.log(1 + test_df['surge_pct'])
test_df['weekday_pct_log'] = np.log(1 + test_df['weekday_pct'])
test_df['miles_log'] = np.log(1 + test_df['miles'])

In [17]:
# x_train, x_test, y_train, y_test = train_test_split(df.drop('target',axis=1), df['target'], test_size = 0.3)

In [18]:
model0 = ['constant','luxury_car_user']
model1 = ['constant','luxury_car_user', 'trips_in_first_30_days']
model2 = ['constant','luxury_car_user', 'trips_in_first_30_days', 'LTV']
model3 = ['constant','luxury_car_user', 'trips_in_first_30_days', 'LTV', 'miles']
model4 = ['constant','luxury_car_user', 'trips_in_first_30_days', 'LTV', 'miles', 'avg_ride_price']
model5 = ['constant','luxury_car_user', 'trips_in_first_30_days', 'LTV', 'miles', 'avg_ride_price', 'avg_rating_of_driver_imputed']

model6 = ['constant','luxury_car_user', 'trips_in_first_30_days', 'LTV', 'miles', 'avg_ride_price', 'avg_rating_of_driver_imputed','Astapor', 'King\'s Landing']
model7 = ['constant','luxury_car_user', 'trips_in_first_30_days', 'LTV', 'miles', 'avg_ride_price', 'avg_rating_of_driver_imputed','Astapor', 'King\'s Landing', 'iPhone']
model8 = ['constant','luxury_car_user', 'trips_in_first_30_days', 'LTV', 'miles', 'avg_ride_price', 'avg_rating_of_driver_imputed','Astapor', 'King\'s Landing', 'iPhone', 'LTV_log']
model9 = ['constant','luxury_car_user', 'trips_in_first_30_days', 'LTV', 'miles', 'avg_ride_price_log', 'avg_rating_of_driver_imputed','Astapor', 'King\'s Landing', 'iPhone', 'LTV_log']
model10 = ['constant','luxury_car_user', 'trips_in_first_30_days', 'LTV', 'miles', 'avg_ride_price_log', 'avg_rating_of_driver_imputed','Astapor', 'King\'s Landing', 'iPhone', 'LTV_log','surge_pct', 'weekday_pct']
model11 = ['constant','luxury_car_user', 'LTV','trips_in_first_30_days', 'miles', 'avg_ride_price_log', 'avg_rating_of_driver_imputed','Astapor', 'King\'s Landing', 'iPhone', 'LTV_log','surge_pct', 'weekday_pct', 'surge_pct_log']
model12 = ['constant','luxury_car_user', 'LTV','trips_in_first_30_days', 'miles', 'avg_ride_price_log', 'avg_rating_of_driver_imputed','Astapor', 'King\'s Landing', 'iPhone', 'LTV_log', 'weekday_pct', 'surge_pct_log', 'surge_pct']
model13 = ['constant','luxury_car_user', 'LTV','trips_in_first_30_days', 'miles', 'avg_ride_price_log', 'avg_rating_of_driver_imputed','Astapor', 'King\'s Landing', 'iPhone', 'LTV_log', 'weekday_pct', 'surge_pct_log', 'surge_pct', 'weekday_pct_log']
model14 = ['constant','luxury_car_user', 'LTV','trips_in_first_30_days', 'miles', 'avg_ride_price_log', 'avg_rating_of_driver_imputed','Astapor', 'King\'s Landing', 'iPhone', 'LTV_log', 'weekday_pct', 'surge_pct_log', 'surge_pct', 'weekday_pct_log', 'miles_log']

model15 = ['constant','luxury_car_user', 'trips_in_first_30_days', 'weekday_pct', 'iPhone','surge_pct', "King's Landing", 'Astapor']
model16 = ['luxury_car_user', 'LTV','trips_in_first_30_days', 'miles', 'avg_ride_price_log', 'avg_rating_of_driver_imputed','Astapor', 'King\'s Landing', 'iPhone', 'LTV_log', 'weekday_pct', 'surge_pct_log', 'surge_pct', 'weekday_pct_log']

features_list =[model0, model1, model2, model3, model4, model5, model6, model7, model8, model9, model10, model11, model12, model13, model14, model15, model16]

In [19]:
y = df.target
df.drop('target', 1, inplace = True)

In [20]:
model2 = RandomForestClassifier(n_estimators= 36, min_samples_split= 18, min_samples_leaf = 24, max_depth = 15)
# model2 = RandomForestClassifier(n_estimators= 29, min_samples_split = 32, min_samples_leaf = 6, max_depth =  14)

model2.fit(df[model13], y);


In [21]:
test_y = test_df.target
test_df.drop('target',1,inplace = True)

In [22]:
y_pred = model2.predict(test_df[model13])

In [27]:
y_pred_prob = model2.predict_proba(test_df[model13])[:,1]

In [28]:
print('Precision score', precision_score(test_y, y_pred))
print('Recall score',recall_score(test_y, y_pred))
print('Accuracy score',accuracy_score(test_y, y_pred))
print('AUC score', roc_auc_score(test_y, y_pred_prob))
print('Confusion Matrix\n', confusion_matrix(test_y, y_pred))

Precision score 0.7933859675256965
Recall score 0.8551701991008349
Accuracy score 0.7711
AUC score 0.8418174498093309
Confusion Matrix
 [[2385 1387]
 [ 902 5326]]


# Finished! Bada Boom!!

In [45]:
logit_model_loop(df,y_train,x_test,y_test,features_list)

NameError: name 'y_train' is not defined

In [ ]:
logit_model_loop(x_train,y_train,x_test,y_test,features_list)

Original - with outliers

accuracy with model 0:  0.6355
precision with model 0:  0.7119209718328661
recall with model 0:  0.706451185587495
 
accuracy with model 1:  0.67575
precision with model 1:  0.9297824055533307
recall with model 1:  0.6742497579864473
 
accuracy with model 2:  0.6705833333333333
precision with model 2:  0.9277800026698705
recall with model 2:  0.6707199382358618
 
accuracy with model 3:  0.67025
precision with model 3:  0.9268455479909224
recall with model 3:  0.6706916537867079
 
accuracy with model 4:  0.6741666666666667
precision with model 4:  0.9142971565879054
recall with model 4:  0.6769793417020856
 
accuracy with model 5:  0.6729166666666667
precision with model 5:  0.9137631824856495
recall with model 5:  0.6761161596207033
 
accuracy with model 6:  0.7035833333333333
precision with model 6:  0.8703777866773461
recall with model 6:  0.7160114210410718
 
accuracy with model 7:  0.71875
precision with model 7:  0.8511547189961287
recall with model 7:  0.7383047707271885
 
accuracy with model 8:  0.7241666666666666
precision with model 8:  0.8531571218795888
recall with model 8:  0.7430531333565864
 
accuracy with model 9:  0.72625
precision with model 9:  0.8564944600186891
recall with model 9:  0.7437978205425458
 

changed - without outliers

accuracy with model 0:  0.63275
precision with model 0:  0.7090884682507719
recall with model 0:  0.7022068598776922
 
accuracy with model 1:  0.6715
precision with model 1:  0.9276412941334408
recall with model 1:  0.6700281198487346
 
accuracy with model 2:  0.66925
precision with model 2:  0.9292522486239764
recall with model 2:  0.6678888460054033
 
accuracy with model 3:  0.6703333333333333
precision with model 3:  0.9304604644918781
recall with model 3:  0.6684347574500916
 
accuracy with model 4:  0.6731666666666667
precision with model 4:  0.9182440596053162
recall with model 4:  0.6736925046784201
 
accuracy with model 5:  0.6725833333333333
precision with model 5:  0.9152906430393342
recall with model 5:  0.6739818109924871
 
accuracy with model 6:  0.7021666666666667
precision with model 6:  0.8687072090213451
recall with model 6:  0.7136869968015882
 
accuracy with model 7:  0.7175
precision with model 7:  0.8552825882668814
recall with model 7:  0.7337325809052171
 
accuracy with model 8:  0.7200833333333333
precision with model 8:  0.8582360048328634
recall with model 8:  0.735165593376265
 
accuracy with model 9:  0.7203333333333334
precision with model 9:  0.8618606524365686
recall with model 9:  0.7339659311764033

In [27]:
# model = RandomForestClassifier()
# model.fit(x_train[model7], y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [110]:
n_estimators_range = range(1,50)
min_samples_split_range = range(2,50)
min_samples_leaf_range = range(2,50)
max_depth_range = range(1,20)

param_grid = dict({
    'n_estimators': n_estimators_range,
    'min_samples_split': min_samples_split_range,
    'min_samples_leaf': min_samples_leaf_range,
    'max_depth': max_depth_range,
})

search = RandomizedSearchCV(model, param_grid, cv = 10, scoring = 'accuracy')
search.fit(x_train[model13], y_train)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': range(1, 50), 'min_samples_split': range(2, 50), 'min_samples_leaf': range(2, 50), 'max_depth': range(1, 20)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [111]:
print(search.best_score_)
print(search.best_params_)
print(search.best_estimator_)

0.7791785714285714
{'n_estimators': 34, 'min_samples_split': 27, 'min_samples_leaf': 12, 'max_depth': 11}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=11, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=12, min_samples_split=27,
            min_weight_fraction_leaf=0.0, n_estimators=34, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


array([0.        , 0.03883705, 0.06375214, 0.14607049, 0.14499842,
       0.3744322 , 0.1021648 , 0.0186103 , 0.06519148, 0.04594313])

In [58]:
model2 = RandomForestClassifier(n_estimators= 36, min_samples_split= 18, min_samples_leaf = 24, max_depth = 15)
# model2 = RandomForestClassifier(n_estimators= 29, min_samples_split = 32, min_samples_leaf = 6, max_depth =  14)

model2.fit(x_train[model13], y_train);


NameError: name 'x_train' is not defined

In [135]:
y_pred = model2.predict(x_test[model13])

In [137]:
print('Precision score', precision_score(y_test, y_pred))
print('Recall score',recall_score(y_test, y_pred))
print('Accuracy score',accuracy_score(y_test, y_pred))
print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

Precision score 0.7965647460104763
Recall score 0.8731472826812658
Accuracy score 0.7816666666666666
Confusion Matrix
 [[2841 1670]
 [ 950 6539]]


In [113]:
adaboost = AdaBoostClassifier()
adaboost.fit(x_train[model13], y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [118]:

n_estimators_range = range(1,50)
learning_rate_range = list(np.arange(0,1,0.05))

param_grid = dict({
    'n_estimators': n_estimators_range,
    'learning_rate': learning_rate_range,
})

search = RandomizedSearchCV(adaboost, param_grid, cv = 10, scoring = 'accuracy')
search.fit(x_train[model13], y_train)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': range(1, 50), 'learning_rate': [0.0, 0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.30000000000000004, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.65, 0.7000000000000001, 0.75, 0.8, 0.8500000000000001, 0.9, 0.9500000000000001]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [23]:
GB = GradientBoostingClassifier()
GB.fit(df[model13], y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [25]:
n_estimators_range = range(1,50)
learning_rate_range = list(np.arange(0,1,0.05))

param_grid = dict({
    'n_estimators': n_estimators_range,
    'learning_rate': learning_rate_range,
})

search = RandomizedSearchCV(GB, param_grid, cv = 10, scoring = 'accuracy')
search.fit(df[model13], y)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': range(1, 50), 'learning_rate': [0.0, 0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.30000000000000004, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.65, 0.7000000000000001, 0.75, 0.8, 0.8500000000000001, 0.9, 0.9500000000000001]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scori

In [27]:
print(search.best_score_)
print(search.best_params_)
print(search.best_estimator_)

0.783475
{'n_estimators': 22, 'learning_rate': 0.8}
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.8, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=22,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


In [30]:
GB = GradientBoostingClassifier(n_estimators= 22, learning_rate = 0.8)
GB.fit(df[model13], y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.8, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=22,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [31]:
y_pred = GB.predict(test_df[model13])

In [57]:
print('Precision score', precision_score(test_y, y_pred))
print('Recall score',recall_score(test_y, y_pred))
print('Accuracy score',accuracy_score(test_y, y_pred))
print('Confusion Matrix\n', confusion_matrix(test_y, y_pred))

Precision score 0.793849828332587
Recall score 0.8538856775850996
Accuracy score 0.7709
Confusion Matrix
 [[2391 1381]
 [ 910 5318]]


In [119]:
# print(search.best_score_)
# print(search.best_params_)
# print(search.best_estimator_)

0.7725
{'n_estimators': 42, 'learning_rate': 0.65}
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.65, n_estimators=42, random_state=None)


In [ ]:

# Precision score 0.7965647460104763
# Recall score 0.8731472826812658
# Accuracy score 0.7816666666666666
# Confusion Matrix
#  [[2841 1670]
#  [ 950 6539]]